사용 모델 : EfficientNet-B0


In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import pandas as pd
import numpy as np
import cv2
import os
from PIL import Image
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.utils import shuffle
from tqdm import tqdm
import multiprocessing

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import pandas as pd
import numpy as np
import cv2
import os
from PIL import Image
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.utils import shuffle
from tqdm import tqdm
import multiprocessing

# =========================================================
# [1] 데이터 전처리 및 분할 함수 
# =========================================================
def prepare_datasets(csv_path, target_classes):
    """
    csv_path: 메타데이터 CSV 경로
    target_classes: ['No Finding', 'Disease_A', 'Disease_B', 'Disease_C'] 순서
    """
    print(f"📂 데이터셋 로드 중: {csv_path}")
    df = pd.read_csv(csv_path)
    
    # 1. 각 클래스별 데이터프레임 추출
    dfs = {}
    for label in target_classes:
        # 해당 텍스트를 포함하는 행 필터링
        dfs[label] = df[df['Finding Labels'].str.contains(label)]
    
    # 2. 데이터 개수 기준점 설정 (가장 적은 질병 데이터 수 기준)
    # No Finding을 제외한 나머지 질병들 중 최소 데이터 수 확인
    disease_counts = [len(dfs[c]) for c in target_classes if c != 'No Finding']
    if not disease_counts:
        raise ValueError("질병 클래스 데이터가 없습니다.")
        
    min_count = min(disease_counts)
    
    # 3. 비율 설정 (No Finding 2 : 질병 1)
    n_disease = min_count
    n_no_finding = min_count * 2
    
    print(f"📊 Sampling Base Count (1 ratio): {n_disease}")
    print(f"📊 No Finding Count (2 ratio): {n_no_finding}")
    
    # 4. 샘플링 (Train Set 구성)
    train_fragments = []
    for label in target_classes:
        n_sample = n_no_finding if label == 'No Finding' else n_disease
        
        # 데이터가 부족할 경우를 대비해 n_sample 조정 (안전장치)
        actual_count = len(dfs[label])
        if actual_count < n_sample:
            print(f"⚠️ Warning: {label} 데이터 부족 ({actual_count} < {n_sample}). 전체 사용.")
            n_sample = actual_count
            
        # 랜덤 샘플링
        sampled = dfs[label].sample(n=n_sample, random_state=42)
        train_fragments.append(sampled)
        
    # 학습 데이터 병합 및 섞기
    train_df = pd.concat(train_fragments)
    train_df = shuffle(train_df, random_state=42).reset_index(drop=True)
    
    # 5. 테스트 데이터 생성 (학습에 사용되지 않은 나머지 데이터)
    # 이미지 인덱스(파일명)를 기준으로 차집합 연산
    train_filenames = set(train_df['Image Index'].values)
    
    # 전체 데이터 중 학습에 안 쓰인 것만 필터링
    test_df_all = df[~df['Image Index'].isin(train_filenames)]
    
    # 테스트셋에서도 우리가 관심 있는 4개 클래스 관련 데이터만 남김 (필요 시 수정 가능)
    condition = test_df_all['Finding Labels'].str.contains(target_classes[0])
    for label in target_classes[1:]:
        condition |= test_df_all['Finding Labels'].str.contains(label)
        
    test_df = test_df_all[condition].reset_index(drop=True)
    
    print(f"✅ Train Set Completed: {len(train_df)} images")
    print(f"✅ Test Set Completed : {len(test_df)} images")
    
    return train_df, test_df

# =========================================================
# [2] 전략 B (Masking & Cropping) Dataset 클래스
# =========================================================
class LungMaskDataset(Dataset):
    def __init__(self, df, img_dir, mask_dir, classes, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.classes = classes
        self.df['labels_list'] = self.df['Finding Labels'].apply(lambda x: x.split('|'))

    def __len__(self):
        return len(self.df)
    
    def apply_mask_strategy(self, img_path, mask_path):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        if img is None: return None
        # 마스크가 없으면 원본 그대로 사용
        if mask is None: return Image.fromarray(img).convert('RGB')
        
        # 크기 맞추기
        if img.shape != mask.shape:
            mask = cv2.resize(mask, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)
            
        # 1. 배경 제거 (Masking)
        masked_img = cv2.bitwise_and(img, img, mask=mask)
        
        # 2. 폐 영역 Cropping
        points = cv2.findNonZero(mask)
        if points is not None:
            x, y, w, h = cv2.boundingRect(points)
            crop_img = masked_img[y:y+h, x:x+w]
        else:
            crop_img = masked_img
            
        return Image.fromarray(crop_img).convert('RGB')

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        fname = row['Image Index']
        
        img_path = os.path.join(self.img_dir, fname)
        mask_path = os.path.join(self.mask_dir, fname)
        
        image = self.apply_mask_strategy(img_path, mask_path)
        
        if image is None: # 로드 실패 시 검은 이미지
            image = Image.new('RGB', (224, 224))
            
        if self.transform:
            image = self.transform(image)
            
        # Multi-hot Labeling
        label_vec = torch.zeros(len(self.classes), dtype=torch.float32)
        for i, cls_name in enumerate(self.classes):
            if cls_name in row['labels_list']:
                label_vec[i] = 1.0
                
        return image, label_vec

# =========================================================
# [3] 모델 및 학습 함수
# =========================================================
def get_efficientnet_model(num_classes, device):
    model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, num_classes)
    return model.to(device)

def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, targets in tqdm(loader, desc="Training", leave=False):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(loader)

def evaluate(model, loader, device, target_names):
    model.eval()
    all_targets = []
    all_preds = []
    with torch.no_grad():
        for inputs, targets in tqdm(loader, desc="Evaluating", leave=False):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            preds = torch.sigmoid(outputs)
            all_targets.append(targets.cpu().numpy())
            all_preds.append(preds.cpu().numpy())
    
    all_targets = np.vstack(all_targets)
    all_preds = np.vstack(all_preds)
    
    try:
        auc = roc_auc_score(all_targets, all_preds, average='macro')
    except:
        auc = 0.0
    return auc, all_targets, all_preds

# =========================================================
# [Main Execution Flow]
# =========================================================
if __name__ == "__main__":
    # --- 설정값 (사용자 경로 수정 필수) ---
    RAW_IMG_DIR = "D:/lung_xray/final_denoised"          # 원본 이미지 폴더
    MASK_DIR = "D:/lung_xray/final_contour_masks" # 마스크 폴더
    CSV_PATH = "C:/project/lungX/CXR8/Data_Entry_processed_Final.csv"        # CSV 파일 경로
    
    # 4개 클래스 정의
    TARGET_CLASSES = ['No Finding', 'Infiltration', 'Effusion', 'Atelectasis'] 
    
    BATCH_SIZE = 5 #GPU가 없어서... 10에서 변경
    EPOCHS = 5
    LR = 1e-4
    
    # 1. 데이터 분할 함수 호출 (여기서 NameError 해결!)
    train_df, test_df = prepare_datasets(CSV_PATH, TARGET_CLASSES)
    
    # 2. DataLoader 설정
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    train_ds = LungMaskDataset(train_df, RAW_IMG_DIR, MASK_DIR, TARGET_CLASSES, transform)
    test_ds = LungMaskDataset(test_df, RAW_IMG_DIR, MASK_DIR, TARGET_CLASSES, transform)
    
    num_workers = min(8, multiprocessing.cpu_count())
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    
    # 3. 모델 준비 및 학습
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🚀 Device: {device}")
    
    model = get_efficientnet_model(len(TARGET_CLASSES), device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    
    print("🚀 Training Start...")
    for epoch in range(EPOCHS):
        loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
        auc, _, _ = evaluate(model, test_loader, device, TARGET_CLASSES)
        print(f"Epoch [{epoch+1}/{EPOCHS}] Loss: {loss:.4f} | Test AUC: {auc:.4f}")
        
    print("\n[Final Report]")
    final_auc, y_true, y_pred = evaluate(model, test_loader, device, TARGET_CLASSES)
    print(f"Overall Macro AUC: {final_auc:.4f}")
    
    for i, cls in enumerate(TARGET_CLASSES):
        try:
            cls_auc = roc_auc_score(y_true[:, i], y_pred[:, i])
            print(f" - {cls:<15} AUC: {cls_auc:.4f}")
        except:
            pass

📂 데이터셋 로드 중: C:/project/lungX/CXR8/Data_Entry_processed_Final.csv
📊 Sampling Base Count (1 ratio): 11203
📊 No Finding Count (2 ratio): 22406
✅ Train Set Completed: 56015 images
✅ Test Set Completed : 41251 images
🚀 Device: cpu
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\KDT-31/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [05:05<00:00, 70.2kB/s] 


🚀 Training Start...


Training:   0%|          | 0/3501 [00:00<?, ?it/s]